In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/company-stock-and-investment/Company Stock and Investment.csv',index_col=['Date'], parse_dates=['Date'])

In [ ]:
import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')

import numpy as np                               # vectors and matrices
import pandas as pd                              # tables and data manipulations
import matplotlib.pyplot as plt                  # plots
import seaborn as sns                            # more plots

from dateutil.relativedelta import relativedelta # working with dates with style
from scipy.optimize import minimize              # for function minimization

import statsmodels.formula.api as smf            # statistics and econometrics
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from itertools import product                    # some useful functions
from tqdm import tqdm_notebook

In [ ]:
df=pd.read_csv('/kaggle/input/company-stock-and-investment/Company Stock and Investment.csv')

In [ ]:
df['Date']=pd.to_datetime(df['Date'])

In [ ]:
df1=df

In [ ]:
df1.sort_values(by='Date')

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df1=df.tail(200)

In [ ]:
df1

In [ ]:
df1

In [ ]:
df1=df1.set_index(df1['Date']).drop('Date', 1)

In [ ]:
df1['Other sharesInvestments'].plot()

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(df1['Other sharesInvestments'])
plt.title('Other sharesInvestments')
plt.grid(True)
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
df1=df1.drop(['Oil Investments','Gold Investments','Comp Stock'],axis=1)

In [ ]:
df1

In [ ]:
print("Min:",df1.index.min())
print("Max:",df1.index.max())

In [ ]:
df1.columns

In [ ]:
plt.figure(figsize=(17,5))
df1['Other sharesInvestments'].plot()
plt.title("Other sharesInvestments",fontsize=20)
plt.show()

In [ ]:
import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')

import numpy as np                               # vectors and matrices
import pandas as pd                              # tables and data manipulations
import matplotlib.pyplot as plt                  # plots
import seaborn as sns                            # more plots

from dateutil.relativedelta import relativedelta # working with dates with style
from scipy.optimize import minimize              # for function minimization

import statsmodels.formula.api as smf            # statistics and econometrics
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from itertools import product                    # some useful functions
from tqdm import tqdm_notebook



%matplotlib inline

In [ ]:
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
def moving_average(series, n):
    """
        Calculate average of last n observations
    """
    return np.average(series[-n:])

In [ ]:
def plotMovingAverage(series, window, plot_intervals=False, scale=1.96, plot_anomalies=False):

    """
        series - dataframe with timeseries
        window - rolling window size 
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 

    """
    rolling_mean = series.rolling(window=window).mean()

    plt.figure(figsize=(15,5))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    # Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bond = rolling_mean - (mae + scale * deviation)
        upper_bond = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bond, "r--", label="Upper Bond / Lower Bond")
        plt.plot(lower_bond, "r--")
        
        # Having the intervals, find abnormal values
        if plot_anomalies:
            anomalies = pd.DataFrame(index=series.index, columns=series.columns)
            anomalies[series<lower_bond] = series[series<lower_bond]
            anomalies[series>upper_bond] = series[series>upper_bond]
            plt.plot(anomalies, "ro", markersize=10)
        
    plt.plot(series[window:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)

In [ ]:
df1

In [ ]:
plotMovingAverage(df1, 12) 

In [ ]:
plotMovingAverage(df1, 1) 

In [ ]:
plotMovingAverage(df1, 24) 

In [ ]:
plotMovingAverage(df['Other sharesInvestments'], 4, plot_intervals=True)

In [ ]:
def weighted_average(series, weights):
    """
        Calculate weighter average on series
    """
    result = 0.0
    weights.reverse()
    for n in range(len(weights)):
        result += series.iloc[-n-1] * weights[n]
    return float(result)

In [ ]:
weighted_average(df1, [0.6, 0.3, 0.1])

In [ ]:
def exponential_smoothing(series, alpha):
    """
        series - dataset with timestamps
        alpha - float [0.0, 1.0], smoothing parameter
    """
    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result

In [ ]:
def plotExponentialSmoothing(series, alphas):
    """
        Plots exponential smoothing with different alphas
        
        series - dataset with timestamps
        alphas - list of floats, smoothing parameters
        
    """
    with plt.style.context('seaborn-white'):    
        plt.figure(figsize=(15, 7))
        for alpha in alphas:
            plt.plot(exponential_smoothing(series, alpha), label="Alpha {}".format(alpha))
        plt.plot(series.values, "c", label = "Actual")
        plt.legend(loc="best")
        plt.axis('tight')
        plt.title("Exponential Smoothing")
        plt.grid(True);

In [ ]:
plotExponentialSmoothing(df1['Other sharesInvestments'], [0.3, 0.05])

In [ ]:
df1.columns

In [ ]:
def double_exponential_smoothing(series, alpha, beta):
    """
        series - dataset with timeseries
        alpha - float [0.0, 1.0], smoothing parameter for level
        beta - float [0.0, 1.0], smoothing parameter for trend
    """
    # first value is same as series
    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)
    return result

def plotDoubleExponentialSmoothing(series, alphas, betas):
    """
        Plots double exponential smoothing with different alphas and betas
        
        series - dataset with timestamps
        alphas - list of floats, smoothing parameters for level
        betas - list of floats, smoothing parameters for trend
    """
    
    with plt.style.context('seaborn-white'):    
        plt.figure(figsize=(20, 8))
        for alpha in alphas:
            for beta in betas:
                plt.plot(double_exponential_smoothing(series, alpha, beta), label="Alpha {}, beta {}".format(alpha, beta))
        plt.plot(series.values, label = "Actual")
        plt.legend(loc="best")
        plt.axis('tight')
        plt.title("Double Exponential Smoothing")
        plt.grid(True)

In [ ]:
plotDoubleExponentialSmoothing(df['Other sharesInvestments'], alphas=[0.9, 0.02], betas=[0.9, 0.02])

In [ ]:
plotDoubleExponentialSmoothing(df1['Other sharesInvestments'], alphas=[0.9, 0.02], betas=[0.9, 0.02])

In [ ]:
#Ho: It is non stationary
#H1: It is stationary

def adfuller_test(sales):
    result=adfuller(sales) # same as above ,variable name changed
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data  is stationary")
    else:
        print("weak evidence against null hypothesis, time series is non-stationary ")
    

In [ ]:
adfuller_test(df['Other sharesInvestments'])

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from subprocess import check_output
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error

In [ ]:
df=pd.read_csv('/kaggle/input/company-stock-and-investment/Company Stock and Investment.csv')

In [ ]:
df[['Other sharesInvestments']].plot()
plt.title("company")
plt.show()

In [ ]:
dr = df.cumsum()
dr.plot()
plt.title('Cumulative Returns')

In [ ]:
plt.figure(figsize=(10,10))
lag_plot(df['Other sharesInvestments'], lag=1)
plt.title('Autocorrelation plot')

In [ ]:
df1=df.drop(['Oil Investments', 'Gold Investments',
       'Comp Stock'],axis=1)

In [ ]:
df1

In [ ]:
df['Date']

In [ ]:
train_data, test_data = df1[0:int(len(df1)*0.8)], df1[int(len(df1)*0.8):]
plt.figure(figsize=(12,7))
plt.title('Prices')
plt.xlabel('Dates')
plt.ylabel('Prices')
plt.plot(df1['Other sharesInvestments'], 'blue', label='Training Data')
plt.plot(test_data['Other sharesInvestments'], 'green', label='Testing Data')
plt.xticks(np.arange(0,1983, 300), df['Date'][0:1983:300])
plt.legend()



In [ ]:
def smape_kun(y_true, y_pred):
    return np.mean((np.abs(y_pred - y_true) * 200/ (np.abs(y_pred) + np.abs(y_true))))

In [ ]:
train_ar = train_data['Other sharesInvestments'].values
test_ar = test_data['Other sharesInvestments'].values


history = [x for x in train_ar]
print(type(history))
predictions = list()
for t in range(len(test_ar)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test_ar[t]
    history.append(obs)
    #print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test_ar, predictions)
print('Testing Mean Squared Error: %.3f' % error)
error2 = smape_kun(test_ar, predictions)
print('Symmetric mean absolute percentage error: %.3f' % error2)

In [ ]:
plt.figure(figsize=(12,7))
plt.plot(df1['Other sharesInvestments'], 'green', color='blue', label='Training Data')
plt.plot(test_data.index, predictions, color='green', marker='o', linestyle='dashed', 
         label='Predicted Price')
plt.plot(test_data.index, test_data['Other sharesInvestments'], color='red', label='Actual Price')
plt.title('Prices Prediction')
plt.xlabel('Dates')
plt.ylabel('Prices')
plt.xticks(np.arange(0,1983, 300), df['Date'][0:1983:300])
plt.legend()

In [ ]:
def getWeights(d,lags):
    # return the weights from the series expansion of the differencing operator
    # for real orders d and up to lags coefficients
    w=[1]
    for k in range(1,lags):
        w.append(-w[-1]*((d-k+1))/k)
    w=np.array(w).reshape(-1,1) 
    return w
def plotWeights(dRange, lags, numberPlots):
    weights=pd.DataFrame(np.zeros((lags, numberPlots)))
    interval=np.linspace(dRange[0],dRange[1],numberPlots)
    for i, diff_order in enumerate(interval):
        weights[i]=getWeights(diff_order,lags)
    weights.columns = [round(x,2) for x in interval]
    fig=weights.plot(figsize=(15,6))
    plt.legend(title='Order of differencing')
    plt.title('Lag coefficients for various orders of differencing')
    plt.xlabel('lag coefficients')
    #plt.grid(False)
    plt.show()
def ts_differencing(series, order, lag_cutoff):
    # return the time series resulting from (fractional) differencing
    # for real orders order up to lag_cutoff coefficients
    
    weights=getWeights(order, lag_cutoff)
    res=0
    for k in range(lag_cutoff):
        res += weights[k]*series.shift(k).fillna(0)
    return res[lag_cutoff:] 
plotWeights([0.1,0.9],20,5)

In [ ]:
plt.figure(figsize=(12,7))
plt.plot(test_data.index, predictions, color='black', marker='o', linestyle='dashed', 
         label='Predicted Price')
plt.plot(test_data.index, test_data['Other sharesInvestments'], color='red', label='Actual Price')
plt.xticks(np.arange(1486,1983, 60), df['Date'][1486:1983:60])
plt.title('  Prediction')
plt.xlabel('Dates')
plt.ylabel('Prices')
plt.legend()

In [ ]:
train_data = df1
plt.figure(figsize=(12,7))
plt.title('Prices')
plt.xlabel('Dates')
plt.ylabel('Prices')
plt.plot(df1['Other sharesInvestments'], 'blue', label='Training Data')
plt.xticks(np.arange(0,1983, 300), df['Date'][0:1983:300])
plt.legend()



In [ ]:
pred_model=pd.Series(model_fit.fittedvalues, copy=True)

In [ ]:
model_fit.plot_predict(1,2020)

In [ ]:
print(pred_model.head())

In [ ]:
pred_model_cumsum =pred_model.cumsum()
print(pred_model_cumsum)

In [ ]:
pred_model_log =pd.Series(df1['Other sharesInvestments'],index=df.index)


In [ ]:
pred_model_log =pred_model_log.add(pred_model_cumsum,fill_value=0)
pred_model_log.head()

In [ ]:
pred =np.exp(pred_model_log)
plt.plot(df1['Other sharesInvestments'])
plt.plot(pred)

In [ ]:
x=model_fit.forecast(steps=36)
print(x)